In [1]:
from datetime import datetime
import pickle

In [7]:
datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

'30-10-2020_17-21-30'

In [2]:
with open('Data/traj30-10-2020_17-26-15.pkl','rb') as f:
    traj = pickle.load(f)

In [3]:
traj

{'loc': [array([0., 0.]),
  array([0., 0.]),
  array([0., 0.]),
  array([0., 0.]),
  array([1658.,  205.]),
  array([0., 0.]),
  array([0., 0.]),
  array([0., 0.]),
  array([0., 0.]),
  array([0., 0.])],
 't': [0.39099999982863665,
  0.5779999997466803,
  0.7659999998286366,
  0.9529999997466803,
  1.1409999998286366,
  1.3279999997466803,
  1.5159999998286366,
  1.6880000000819564,
  1.875,
  2.0469999997876585],
 'endpts': [array([[-1., -1.],
         [-1., -1.]]),
  array([[-1., -1.],
         [-1., -1.]]),
  array([[-1., -1.],
         [-1., -1.]]),
  array([[-1., -1.],
         [-1., -1.]]),
  array([[1641, 1651],
         [ 193,  209]]),
  array([[-1., -1.],
         [-1., -1.]]),
  array([[-1., -1.],
         [-1., -1.]]),
  array([[-1., -1.],
         [-1., -1.]]),
  array([[-1., -1.],
         [-1., -1.]]),
  array([[-1., -1.],
         [-1., -1.]])],
 'obs': [array([0., 0.]),
  array([0., 0.]),
  array([0., 0.]),
  array([0., 0.]),
  array([0.16666667, 0.83333333]),
  array([